# 멜론차트 크롤링

In [293]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
url = "https://www.melon.com/chart/"
header = {'User-Agent':'Mozilla/5.0 (Windows NT 6.3; Trident/7.0; rv:11.0) like Gecko'}
req = requests.get(url, headers=header)
soup = BeautifulSoup(req.text, 'html.parser')

## 1. 필요한 데이터 찾기

In [283]:
# 곡들이 담겨져 있는 태그를 리스트로 저장
song_list = soup.find('table').select('tr.lst50') + soup.find('table').select('tr.lst100')

[<tr class="lst50" data-song-no="35945927" id="lst50">
 <td><div class="wrap t_right"><input class="input_check" name="input_check" title="Ditto 곡 선택" type="checkbox" value="35945927"/></div></td>
 <td><div class="wrap t_center"><span class="rank">1</span><span class="none">위</span></div></td>
 <!-- 차트순위 추가 -->
 <td><div class="wrap">
 <span class="rank_wrap" title="순위 동일">
 <span class="bullet_icons rank_static"><span class="none">순위 동일</span></span>
 <span class="none">0</span>
 </span>
 </div></td>
 <td><div class="wrap">
 <a class="image_typeAll" href="javascript:melon.link.goAlbumDetail('11127145');" title="NewJeans 'OMG'">
 <img alt="NewJeans 'OMG' - 페이지 이동" height="60" onerror="WEBPOCIMG.defaultAlbumImg(this);" src="https://cdnimg.melon.co.kr/cm2/album/images/111/27/145/11127145_20230102135733_500.jpg/melon/resize/120/quality/80/optimize" width="60">
 <span class="bg_album_frame"></span>
 </img></a>
 </div></td>
 <td><div class="wrap">
 <a class="btn button_icons type03 song_inf

In [284]:
# 첫 곡을 가져오기
song = song_list[0]

In [285]:
# 순위
rank = int(song.find_all('td')[1].select_one('.rank').text)
rank

1

In [286]:
# 제목
title = song.find_all('td')[5].select_one('.rank01').find('a').text
title

'Ditto'

In [287]:
# 아티스트 이름
artist = song.find_all('td')[5].select_one('.rank02').find('a').text
artist

'NewJeans'

In [288]:
# 앨범명
album = song.find_all('td')[6].select_one('.rank03').find('a').text
album

"NewJeans 'OMG'"

## 2. 리스트에 저장

In [289]:
chart = []
for song in song_list:
    rank = int(song.find_all('td')[1].select_one('.rank').text) # 순위
    title = song.find_all('td')[5].select_one('.rank01').find('a').text # 제목
    artist = song.find_all('td')[5].select_one('.rank02').find('a').text # 아티스트 이름
    album = song.find_all('td')[6].select_one('.rank03').find('a').text # 앨범명
    chart.append({
        '순위' : rank, '제목' : title, '가수' : artist, '앨범' : album
    })

## 3. 데이터 프레임으로 변환

In [290]:
df_melon = pd.DataFrame(chart)

In [291]:
df_melon

,순위,제목,가수,앨범
0,1,Ditto,NewJeans,NewJeans 'OMG'
1,2,OMG,NewJeans,NewJeans 'OMG'
2,3,Hype boy,NewJeans,NewJeans 1st EP 'New Jeans'
3,4,사건의 지평선,윤하 (YOUNHA),YOUNHA 6th Album Repackage 'END THEORY : Final...
4,5,ANTIFRAGILE,LE SSERAFIM (르세라핌),ANTIFRAGILE
...,...,...,...,...
95,96,Off My Face,Justin Bieber,Justice (The Complete Edition)
96,97,나의 목소리로,김호중,나의 목소리로
97,98,이 소설의 끝을 다시 써보려 해,테이,이 소설의 끝을 다시 써보려 해
98,99,문 (Moon),NCT DREAM,Candy - Winter Special Mini Album


## 4. csv 파일로 변환

In [292]:
df_melon.to_csv('멜론차트TOP100.csv')

# 지니차트 크롤링

In [247]:
site = "https://www.genie.co.kr/chart/"
url = f"{site}top200?ditc=D&ymd=20230111&hh=17&rtm=Y&pg={4}"
header = {'User-Agent':'Mozilla/5.0 (Windows NT 6.3; Trident/7.0; rv:11.0) like Gecko'}
req = requests.get(url, headers=header)
soup = BeautifulSoup(req.text, 'html.parser')

## 1. 필요한 데이터 찾기

In [248]:
# 곡들이 담겨져 있는 태그를 리스트로 저장
song_list = soup.select_one('.list-wrap').find('tbody').find_all('tr')

In [269]:
# 한 곡을 가져오기
song = song_list[32]

In [270]:
# 순위
rank = song.select_one('.number').text.split(" ")[0].strip()
rank

'183'

In [271]:
# 제목
title = song.select_one('.info > .title').text.strip().split('19금')[-1].strip()
title

'abcdefu'

In [272]:
# 아티스트 이름
artist = song.select_one('.info > .artist').text.strip()
artist

'GAYLE'

In [273]:
# 앨범명
album = song.select_one('.info > .albumtitle').text.strip()
album

'abcdefu'

## 2. 리스트에 저장

### 2-1. 첫 페이지

In [274]:
chart = []
for song in soup.select_one('.list-wrap').find('tbody').find_all('tr'):
    rank = song.select_one('.number').text.split(" ")[0].strip() # 순위
    title = song.select_one('.info > .title').text.strip().split('19금')[-1].strip() # 제목
    artist = song.select_one('.info > .artist').text.strip() # 아티스트 이름
    album = song.select_one('.info > .albumtitle').text.strip() # 앨범명
    chart.append({
        '순위' : rank, '제목' : title, '가수' : artist, '앨범' : album
    })

In [228]:
df1 = pd.DataFrame(chart)
df1

,순위,제목,가수,앨범
0,51,FEARLESS,LE SSERAFIM (르세라핌),FEARLESS
1,52,ELEVEN,IVE (아이브),ELEVEN
2,53,The Drum,Alan Walker,"Walkerverse, Pt. I"
3,54,그대를 알고,송하예,그대를 알고
4,55,Stay,The Kid LAROI & Justin Bieber,Stay
5,56,정이라고 하자 (Feat. 10CM),BIG Naughty (서동현),정이라고 하자
6,57,Made You Look,Meghan Trainor,Takin' It Back
7,58,잘가요,주호,잘가요
8,59,딱 10CM만,10CM & BIG Naughty (서동현),딱 10CM만
9,60,우린 그렇게 사랑해서,강민경 (다비치) & 잔나비 최정훈,우린 그렇게 사랑해서


### 2-2. 모든 페이지

In [275]:
chart = []
for i in range(1, 5): # 5 페이지 모두 불러오기
    site = "https://www.genie.co.kr/chart/"
    url = f"{site}top200?ditc=D&ymd=20230111&hh=17&rtm=Y&pg={i}"
    header = {'User-Agent':'Mozilla/5.0 (Windows NT 6.3; Trident/7.0; rv:11.0) like Gecko'}
    req = requests.get(url, headers=header)
    soup = BeautifulSoup(req.text, 'html.parser')
    
    for song in soup.select_one('.list-wrap').find('tbody').find_all('tr'):
        rank = song.select_one('.number').text.split(" ")[0].strip() # 순위
        title = song.select_one('.info > .title').text.strip().split('19금')[-1].strip() # 제목
        artist = song.select_one('.info > .artist').text.strip() # 아티스트 이름
        album = song.select_one('.info > .albumtitle').text.strip() # 앨범명
        chart.append({
            '순위' : rank, '제목' : title, '가수' : artist, '앨범' : album
        })

## 3. 데이터 프레임으로 변환

In [277]:
df_genie = pd.DataFrame(chart)
df_genie.tail(20)

,순위,제목,가수,앨범
180,181,늦은 밤 너의 집 앞 골목길에서,노을,늦은 밤 너의 집 앞 골목길에서
181,182,Way Back Home,숀 (SHAUN),Take
182,183,abcdefu,GAYLE,abcdefu
183,184,너에게 못했던 내 마지막 말은,다비치,너에게 못했던 내 마지막 말은
184,185,시작,가호 (Gaho),이태원 클라쓰 OST Part.2 (JTBC 금토드라마)
185,186,안녕 (Hello),조이 (JOY),안녕 (Hello) - Special Album
186,187,parachute,John K,love ＋ everything else
187,188,O Tannenbaum,김호중,PANORAMA
188,189,WE (Feat. 박재범) (Prod. by Slom),이영지 & 토이고 (toigo) & 제이켠 (J'Kyun) & 잠비노 (Jambin...,쇼미더머니 11 Episode 1
189,190,Light Switch,Charlie Puth,CHARLIE


## 4. csv 파일로 변환

In [279]:
df_genie.to_csv('지니차트TOP100.csv')

# 벅스차트 크롤링

In [2]:
url = "https://music.bugs.co.kr/chart"
header = {'User-Agent':'Mozilla/5.0 (Windows NT 6.3; Trident/7.0; rv:11.0) like Gecko'}
req = requests.get(url, headers=header)
soup = BeautifulSoup(req.text, 'html.parser')

## 1. 필요한 데이터 찾기

In [302]:
# 곡들이 담겨져 있는 태그를 리스트로 저장
song_list = soup.select_one('#CHARTrealtime').find('tbody').find_all('tr')

In [338]:
# 한 곡을 가져오기
song = song_list[19]

In [339]:
# 순위
rank = int(song.select_one('.ranking > strong').text)
rank

20

In [340]:
# 제목
title = song.select_one('.title').text.strip()
title

'Pink Venom'

In [341]:
# 아티스트 이름
artist = song.select_one('.artist').text.strip().split('\n')[0]
artist

'BLACKPINK'

In [342]:
# 앨범명
album = song.select_one('.album').text.strip()
album

'BORN PINK'

## 2. 리스트에 저장

In [343]:
chart = []
for song in soup.select_one('#CHARTrealtime').find('tbody').find_all('tr'):
    rank = int(song.select_one('.ranking > strong').text) # 순위
    title = song.select_one('.title').text.strip() # 제목
    artist = song.select_one('.artist').text.strip().split('\n')[0] # 아티스트 이름
    album = song.select_one('.album').text.strip() # 앨범명
    chart.append({
            '순위' : rank, '제목' : title, '가수' : artist, '앨범' : album
        })

## 3. 데이터 프레임으로 변환

In [345]:
df_bugs = pd.DataFrame(chart)
df_bugs.head(22)

,순위,제목,가수,앨범
0,1,Ditto,NewJeans,NewJeans 'OMG'
1,2,OMG,NewJeans,NewJeans 'OMG'
2,3,Hype Boy,NewJeans,NewJeans 1st EP 'New Jeans'
3,4,사건의 지평선,윤하(Younha/ユンナ),YOUNHA 6th Album Repackage 'END THEORY : Final...
4,5,Attention,NewJeans,NewJeans 1st EP 'New Jeans'
5,6,Candy,NCT DREAM,Candy - Winter Special Mini Album
6,7,ANTIFRAGILE,LE SSERAFIM (르세라핌),ANTIFRAGILE
7,8,After LIKE,IVE (아이브),After LIKE
8,9,NOT SORRY (Feat. pH-1) (Prod. by Slom),이영지,쇼미더머니 11 Episode 3
9,10,WHEN I MOVE,카라(Kara),MOVE AGAIN


## 4. csv 파일로 변환

In [346]:
df_bugs.to_csv('벅스차트TOP100.csv')